Script to work out the info from div3s where the element name is add 

In [1]:
from xml.etree import ElementTree as et

In [2]:
import pandas as pd
import re
import csv

In [4]:
# f = '/Users/deborah/Documents/PHD2018/BM/Markup/Kims/KS_BM_Misc_29_10_2018.xml'
f = 'KS_BM_Misc_05_Nov_2018_DL.xml'

In [5]:
tree = et.parse(f)

In [ ]:
#199a and 199e

In [3]:
def find_div3s(tree):
    results = []
    for div2 in tree.findall('.//*/div2'):
        div2_type = div2.attrib['type']
        if div2_type.startswith('Index'):
            continue
        facs = None
        for child in div2:
            if child.tag == 'figure':
                facs = child.attrib['facs']
            elif child.tag == 'div3':
                results.append((div2_type, facs, child))
            else:
                pass # Ignore everything else
    return results

def find_adds(tree):
    results = []
    
    for div2_type, facs, div3 in find_div3s(tree):
        catnum = None

        def text_in_tag(tag):
            return re.sub(r'\s+', ' ', ''.join(tag.itertext())).strip()
            
        def xml_in_tag(tag):
            # This needs a fix, there is extra stuff at the end
            return re.sub(r'[^>]*$', '', str(et.tostring(tag)))
        
        def process_add(tag, paragraph_text, paragraph_xml):
            rend = tag.attrib.get('rend', None)
            add_text = text_in_tag(tag)
            add_xml = xml_in_tag(tag)
            m = re.match(r'.*(\d{3}.*)', facs)
            if m:
                part_facs = m.group(1)
            else:
                part_facs = facs
            results.append((div2_type, part_facs, catnum, paragraph_text, paragraph_xml, tag.tag, rend, add_text, add_xml))
        
        def process_child(child):
            nonlocal catnum 
            if child.tag == '{http://www.example.org/ns/nonTEI}catnum':
                catnum = text_in_tag(child)
            elif child.tag == 'p':
                paragraph_text = text_in_tag(child)
                paragraph_xml = xml_in_tag(child)
                for e in child.findall('catent/add'):
                    process_add(e, paragraph_text, paragraph_xml)
                for e in child.findall('catent/metamark'):
                    process_add(e, paragraph_text, paragraph_xml)
                for e in child.findall('add'):
                    process_add(e, paragraph_text, paragraph_xml)
                for e in child.findall('metamark'):
                    process_add(e, paragraph_text, paragraph_xml)
            elif child.tag in ['add', 'metamark']:
                process_add(child, '', '')
            elif child.tag in ['illegible', 'handShift', 'strikethrough', 'lb', 'sup']:
                pass # Ignore
            elif child.tag in ['laterhand', 'later-hand']:
                for grandchild in child:
                    process_child(grandchild)
            else:
                print(child.tag)

        for child in div3:
            process_child(child)
        
    return results

with open('KS_BM_Misc_29_10_2018.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['div2 type', 'facs', 'catnum', 'paragraph text', 'paragraph xml', 'tag', 'rend', 'tag text', 'tag xml'])
    for row in find_adds(tree):
        writer.writerow(row)


NameError: name 'tree' is not defined

In [65]:
root = tree.getroot()
root.tag

'test'

In [66]:
for child in root:
    print(child.tag, child.attrib)

teiHeader {}
front {}
body {}


In [67]:
div2 = [x for x in list(root[2]) if x.tag == 'div2']
div2
# dir(list(root[2])[0])
# list(root[2])[0].tag

[<Element 'div2' at 0x11c371ea8>,
 <Element 'div2' at 0x11cf697c8>,
 <Element 'div2' at 0x11d3a0188>,
 <Element 'div2' at 0x11d49cd18>,
 <Element 'div2' at 0x11d524778>,
 <Element 'div2' at 0x11d664db8>,
 <Element 'div2' at 0x11d696e08>,
 <Element 'div2' at 0x11d6ca4a8>,
 <Element 'div2' at 0x11d78e0e8>]

In [88]:
res = {}
# res = [{div2[0].attrib.get('type'): list(x)} for x in div2] 
for x in div2:
    k = x.attrib.get('type')
    if 'Index' in k:
        continue
    res[k] = list(x)
res
# for d in res:
#     r[d.keys()]
# res

{'Miscellanies': [<Element 'figure' at 0x11c371ef8>,
  <Element 'p' at 0x11c371f48>,
  <Element 'p' at 0x11c375048>,
  <Element 'p' at 0x11c375098>,
  <Element 'p' at 0x11c3750e8>,
  <Element 'figure' at 0x11c375188>,
  <Element 'div3' at 0x11c3751d8>,
  <Element 'div3' at 0x11c3754a8>,
  <Element 'div3' at 0x11c3756d8>,
  <Element 'div3' at 0x11c3759f8>,
  <Element 'div3' at 0x11c375c28>,
  <Element 'div3' at 0x11c375ea8>,
  <Element 'div3' at 0x11c3790e8>,
  <Element 'div3' at 0x11c379368>,
  <Element 'div3' at 0x11c379728>,
  <Element 'div3' at 0x11c379c78>,
  <Element 'div3' at 0x11c37e048>,
  <Element 'div3' at 0x11c37e598>,
  <Element 'div3' at 0x11c37e908>,
  <Element 'div3' at 0x11c37ec78>,
  <Element 'div3' at 0x11c385368>,
  <Element 'div3' at 0x11c385728>,
  <Element 'div3' at 0x11c3859a8>,
  <Element 'div3' at 0x11c385c78>,
  <Element 'div3' at 0x11c38c048>,
  <Element 'div3' at 0x11c38c368>,
  <Element 'div3' at 0x11c38c638>,
  <Element 'figure' at 0x11c38c8b8>,
  <Element

In [133]:
df = pd.DataFrame({'catnum': [], 'Div2': [], 'add': [], 'p': []})


out = []
for k,v in res.items():
#     print(k)
    for thing in v:
        if thing.tag == 'div3':
#             print(dir(thing))
#             print(list(thing))
#             print(list(thing)[2].tag)
            for t in list(thing):
                if 'add' in t.tag:
                    out.append(thing)
                    try:
                        df = df.append({'catnum': thing.find('{http://www.example.org/ns/nonTEI}catnum').text,
                                        'Div2': k,
                                        'add': thing.find('add').text,
                                         'p': thing.find('p').text}, 
                                        ignore_index=True)
                    except:
                        pass
    break
len(out)
list(out[0])
df
# out[0].find('{http://www.example.org/ns/nonTEI}catnum').text

,catnum,Div2,add,p
0,-1799.,Miscellanies,252.,\n
1,-1800.,Miscellanies,252.,\n
2,-1801.,Miscellanies,245.,None
3,-1802.,Miscellanies,299 b,None
4,-1802.,Miscellanies,299 b,None
5,-1803.,Miscellanies,226,None
6,-1803.,Miscellanies,226,None
7,-1804.,Miscellanies,245.,None
8,-1805.,Miscellanies,163,None
9,-1805.,Miscellanies,163,None


In [94]:
[<Element '{http://www.example.org/ns/nonTEI}catnum' at 0x11c375228>,
 <Element 'p' at 0x11c375278>,
 <Element 'add' at 0x11c375458>]

In [71]:
# dir(div2[0])
d = div2[0].attrib
d.get('type')
# dir(d)
# dir(div2[0])
# r = d.get('type')

'Miscellanies'

In [36]:
res

[({'type': 'Miscellanies'},
  [<Element 'figure' at 0x11b13e548>,
   <Element 'p' at 0x11b13e598>,
   <Element 'p' at 0x11b13e638>,
   <Element 'p' at 0x11b13e688>,
   <Element 'p' at 0x11b13e6d8>,
   <Element 'figure' at 0x11b13e778>,
   <Element 'div3' at 0x11b13e7c8>,
   <Element 'div3' at 0x11b13ea98>,
   <Element 'div3' at 0x11b13ecc8>,
   <Element 'div3' at 0x11b145048>,
   <Element 'div3' at 0x11b145278>,
   <Element 'div3' at 0x11b1454f8>,
   <Element 'div3' at 0x11b1456d8>,
   <Element 'div3' at 0x11b145958>,
   <Element 'div3' at 0x11b145d18>,
   <Element 'div3' at 0x11b14a2c8>,
   <Element 'div3' at 0x11b14a638>,
   <Element 'div3' at 0x11b14ab88>,
   <Element 'div3' at 0x11b14aef8>,
   <Element 'div3' at 0x11b14f2c8>,
   <Element 'div3' at 0x11b14f958>,
   <Element 'div3' at 0x11b14fd18>,
   <Element 'div3' at 0x11b14ff98>,
   <Element 'div3' at 0x11b1532c8>,
   <Element 'div3' at 0x11b153638>,
   <Element 'div3' at 0x11b153958>,
   <Element 'div3' at 0x11b153c28>,
   <Elem